In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Get user input for stock tickers
tickers_input = input("Enter the stock tickers in your portfolio, separated by commas (e.g., AAPL, MSFT, GOOGL): ")
tickers = [ticker.strip().upper() for ticker in tickers_input.split(',')]

# Define date range for the past 2 years
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)  # Approximately 2 years

# Fetch adjusted close prices for each ticker
data = {}
for ticker in tickers:
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    data[ticker] = stock_data['Adj Close']

# Create a DataFrame of Adjusted Close prices
df = pd.DataFrame(data)

# Handle missing values
df.fillna(method='ffill', inplace=True)  # Forward fill for NaN values
df.dropna(inplace=True)  # Drop remaining NaNs if any

# Calculate daily returns
daily_returns = df.pct_change().dropna()

# Placeholder structure for risk metrics
metrics = {}

# Value at Risk (VaR) Calculation
def calculate_var(daily_returns, confidence_level=0.6):
    """
    Calculate the Value at Risk (VaR) at a specified confidence level.
    """
    var = daily_returns.quantile(1 - confidence_level)
    return var

# Beta Calculation
def calculate_beta(ticker_returns, market_returns):
    """
    Calculate Beta by comparing the stock returns to market returns (S&P 500).
    """
    covariance = ticker_returns.cov(market_returns)
    market_variance = market_returns.var()
    beta = covariance / market_variance
    return beta

# Fetch S&P 500 data as a proxyA for market returns
sp500_data = yf.download('^GSPC', start=start_date, end=end_date)['Adj Close']
market_returns = sp500_data.pct_change().dropna()

# Add risk metrics for each ticker
for ticker in tickers:
    # Calculate VaR
    var = calculate_var(daily_returns[ticker])*100
    
    # Calculate Beta
    beta = calculate_beta(daily_returns[ticker], market_returns)

    # Store metrics for each ticker
    metrics[ticker] = {
        "VaR (60%)": var,
        "Beta": beta,
    }

# Flexible Risk Level Assignment
def assign_risk_level(var, beta):
    """
    Assign a risk level based on VaR and Beta.
    Each metric contributes to a risk score.
    """
    score = 0
    
    # VaR Scoring - High weight (100% importance)
    if var > -0.15:
        score += 1  # Very Low Risk
    elif -0.15 <= var <= -0.35:
        score += 2 # Low Risk
    elif -0.35 <= var < -0.5:
        score += 3  # Moderate Risk
    else:
        score += 4 # High Risk
    
    # Beta Scoring - Lower weight compared to VaR
    if abs(beta) < 0.5:
        score += 0.5  # Lower Beta contributes to lower risk
    elif 0.5 <= abs(beta) <= 1:
        score += 1  # Moderate Beta
    elif 1 <= abs(beta) <= 1.3:
        score += 1.5
    else:
        score += 2  # Higher Beta contributes to higher risk

    # Map total score to risk levels
    if score <= 2:
        return "Very Low Risk"
    elif score <= 4:
        return "Low Risk"
    elif score <= 5.5:
        return "Moderate Risk"
    else:
        return "High Risk"

# Assess risk for each stock
risk_assessment = {}
for ticker, metric_values in metrics.items():
    risk_assessment[ticker] = assign_risk_level(metric_values["VaR (60%)"], metric_values["Beta"])

# Convert metrics and risk assessment to DataFrames for display
metrics_df = pd.DataFrame(metrics).T
risk_assessment_df = pd.DataFrame.from_dict(risk_assessment, orient='index', columns=['Risk Level'])
risk_assessment_df.index.name = 'Ticker'

# Display results
print("\nMetrics for each stock:")
print(metrics_df)

print("\nRisk Assessment for each stock:")
print(risk_assessment_df)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ENEL']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
/var/folders/jh/5qgf89253j945pw6bjvtshgr0000gn/T/ipykernel_92671/3620676299.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # Forward fill for NaN values
[*********************100%***********************]  1 of 1 completed


Metrics for each stock:
      VaR (60%)  Beta
ENEL        NaN   NaN

Risk Assessment for each stock:
       Risk Level
Ticker           
ENEL    High Risk


SyntaxError: invalid syntax (3737097518.py, line 1)